In [1]:
import os
import dlib
import cv2
import time
import numpy as np
from PIL import Image
from scipy.spatial import distance as dist
from collections import OrderedDict
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)


label_map= ["biscuit_a","biscuit_b","biscuit_c"]
label_map.sort()
detect_fn=tf.saved_model.load(".\\saved_model")

In [2]:
detect_fn

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x116b50da788>

In [3]:
import math
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, Conv2D, Flatten 
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
 
# load class model
class_model = load_model('.\\imageacc_model.h5')

#Backbone_SNN
# base_resnet=tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(150,150,3),classes=len(label_map))
# # for i in base_resnet.layers[:(math.ceil(len(base_resnet.layers)/2))]:
# #     i.trainable=False
# conv1=Conv2D(48,(1,1),padding="valid",activation="relu")
# dr1=layers.Dropout(0.3)
# avgpool1 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))
# conv2=Conv2D(78,(1,1),padding="same",activation="relu")
# dr2=layers.Dropout(0.2)
# flatten_layer = Flatten()
# dense1 = Dense(108,activation='sigmoid')

# embedding_model=tf.keras.Sequential([base_resnet,conv1,avgpool1,conv2,flatten_layer,dense1])
# embedding_model.load_weights(".\\checkpoint_94_0.00.hdf5")

In [4]:
class_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 108)               23723690  
                                                                 
 dense_9 (Dense)             (None, 3)                 327       
                                                                 
Total params: 23,724,017
Trainable params: 327
Non-trainable params: 23,723,690
_________________________________________________________________


In [5]:
def detect_boxes(frame, detect_fn,thresh,H,W):
        image_np = np.array(frame)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
        detections = detect_fn(input_tensor)
        # detection_classes should be ints.
        score_mask=(detections['detection_scores']>0.5).numpy().reshape(-1)
        rects_masked=detections['detection_boxes'].numpy().reshape(100,4)[score_mask]
        rects=(rects_masked*np.array([H,W,H,W])).astype(int)
        func=lambda x: (x[1]+x[3])/2
        rect_mask=np.array(list(map(func,rects)))>600*0.30
        print(rects)
        print(rect_mask)
        valid_rects=rects[rect_mask]
        print("Valid_rects\n {}".format(valid_rects))
        return valid_rects
    
def visualize_boxes(image,tracks):    
    for id_,track in tracks.items():
        (startY,startX,endY,endX) = track.box
        name="unknown" if track.name==None else track.name
        # draw the bounding box of the face along with the associated probability
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, name, (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
        #cv2.putText(image, name, (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255,255), 2) 
    return image

In [6]:


# def get_name(model,svm,face_pixels,label_map):
#     img=cv2.resize(face_pixels,(160,160))
#     img = np.around(np.array(img) / 255.0, decimals=12)
#     x_train = np.expand_dims(img, axis=0)
#     embedding = model.predict(x_train)
#     embedding= embedding / np.linalg.norm(embedding, ord=2)
#     name=svm.predict(embedding.reshape(1,-1))
#     return label_map[name]



class Track():
    def __init__(self,tracker,box):
        self.tracker=tracker
        self.box=box
        self.recognized_face=False
        self.encodings=None 
        self.name=None
        self.prob_face={}
        self.centroid=self.get_centroid()
        self.rec_count=0
        #self.spoof_confirm=[]


    def update_track(self,frame):
        self.tracker.update(frame)
        pos=self.tracker.get_position()
        startX = int(pos.left())
        startY = int(pos.top())
        endX = int(pos.right())
        endY = int(pos.bottom())
        tracked_box=np.array([startY,startX,endY,endX])
        print("tracked_box")
        #print(("tracked_box",tracked_box))
        prev_box=np.array(self.box)
        #self.box=tracked_box.astype('int')
        return tracked_box
        #print(("prev_box",prev_box))
        #self.box=self.box if get_overlap(tracked_box,prev_box)>0.95 else tracked_box.astype('int')
        
    def box_recognize(self,image,class_model,label_map,tracker):
        global W,H
        if (self.name==None and 0.46*W<self.get_centroid()[1]<0.50*W):
            starty,startx,endy,endx=self.box
            box_roi = image[starty:endy,startx:endx]  
            img=cv2.cvtColor(box_roi,cv2.COLOR_BGR2RGB)
            im = Image.fromarray(img, 'RGB')
            #Resizing into dimensions you used
            im = im.resize((150,150))
            img_array = (np.array(im).astype(np.float32)/255)
            #Expand dimensions to match the 4 D Tensor shape.
            img_array = np.expand_dims(img_array, axis = 0)
            idx_=class_model.predict(img_array)
            name=label_map[np.argmax(idx_)]
            self.name=name
            tracker.count[name]=tracker.count.get(name,0)+1
            print(tracker.count)
    def get_box(self):
        pos=self.tracker.get_position()
        return [int(pos.left()),int(pos.top()),int(pos.right()),int(pos.bottom())]


    def get_centroid(self):
        box=self.box
        return (int((box[0]+box[2])/2),int((box[1]+box[3])/2))


class Tracker():
    
    global frame
    
    def __init__(self, maxDisappeared=5,maxDistance=150,thresh=None):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.count={}
        self.thresh=thresh
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
        self.maxDistance=maxDistance

    def register(self, rect,frame):
    
        # when registering an object we use the next available object
        # ID to store the centroid
        t = dlib.correlation_tracker()
        (startY, startX, endY, endX)=rect
        drect = dlib.rectangle(startX, startY, endX, endY)
        t.start_track(frame, drect)
        name='obj_'+str(self.nextObjectID)
        self.objects[name] = Track(t,rect)
        self.disappeared[name] = 0
        self.nextObjectID+=1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        trk=self.objects[objectID]
        startY,startX,endY,endX=trk.box
        print("vanishing_arr: {}".format(trk.box))
#         if int(startX/2+endX/2) <self.thresh:     
#             name=trk.name
#             self.count[name]=self.count.get(name,0)+1
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
#         trk=self.objects[objectID]
#         startY,startX,endY,endX=trk.box
#         if int(startX/2+endX/2) <self.thresh:     
#             name=trk.name
#             self.count[name]=self.count.get(name,0)+1
            
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def track_update(self,frame):
        height,width,channel = frame.shape
        for objectID in list(self.disappeared.keys()):
            trk=self.objects[objectID]
            startY,startX,endY,endX=trk.box
            if int(startX/2+endX/2)<(0.35*width):
                self.disappeared[objectID]+=4.7
            if self.disappeared[objectID]>self.maxDisappeared:
                self.deregister(objectID)
        t_rects=[]
        for name in self.objects.keys():
            t_rect=self.objects[name].update_track(frame)
            t_rects.append(t_rect)
        return np.array(t_rects)


    def update(self,frame, coords,thresh):
        
        for objectID in list(self.disappeared.keys()):
            trk=self.objects[objectID]
            if trk.centroid[0]<thresh:
                self.disappeared[objectID]+=1.7
        
        
        
        if len(coords) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects
        inputCentroids = np.zeros((len(coords), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startY, startX, endY, endX)) in enumerate(coords):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # initialize an array of input centroids for the current frame
        
        # if we are currently not tracking any objects take the input
        # centroids and register each of them

        if len(self.objects) == 0:
            for i in range(0, len(coords)):
                self.register(coords[i],frame)
            
            
        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            object_copy=self.objects.copy()
            tracks_copy=list(object_copy.values())
            #print('tracks:'+str(tracks_copy))
            objectIDs = list(object_copy.keys())
            #print('objectIDs:' + str(objectIDs))
            objectCentroids = [i.centroid for i in tracks_copy]

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
            
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # if the distance between centroids is greater than
                # the maximum distance, do not associate the two
                # centroids to the same object
                if D[row, col] > self.maxDistance:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                prev_name=self.objects[objectID].name
                prev_recog_state=self.objects[objectID].recognized_face
                t = dlib.correlation_tracker()
                (y1,x1,y2,x2)=coords[col]
                #name=names[col]
                rect=coords[col]
                drect = dlib.rectangle(int(x1),int(y1),int(x2),int(y2))
                t.start_track(frame, drect)
                n_track=Track(t,rect)
                self.objects[objectID] = n_track
                self.objects[objectID].name=prev_name
                self.objects[objectID].recognized_face=prev_recog_state
                #self.objects[objectID].box = coords[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(coords[col],frame)

        # return the set of trackable objects
        return self.objects


In [8]:
import imutils
import time
from PIL import Image
cap=cv2.VideoCapture(".\\testing_videos\\short_6.mp4")
#if vid_path:
    #cap=cv2.VideoCapture(r'G:\Wondershare UniConverter\Converted\ch01_20210707001338.mp4')
#else:

fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#out = cv2.VideoWriter('full_output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (W,H))
fc=0
thresh=int(600*0.30)
tic=time.time()
name_thresh=600*0.50
tracker=Tracker(maxDisappeared=2,maxDistance=150,thresh=thresh)
func=lambda x: (x[1]+x[3])/2
FPS=[]
while True:
    ret, frame = cap.read()
    if not ret:
        break
    start=time.time()
    img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    im = Image.fromarray(img, 'RGB')
            #Resizing into dimensions you used
    im = im.resize((337,600))
    img_array = (np.array(im).astype(np.float32))
            #Expand dimensions to match the 4 D Tensor shape.
    
    frame=imutils.resize(frame, width=600)
    H,W=frame.shape[:2]
    print("height: {}, width: {}".format(H,W))
    if (fc==0 or fc%4==0):
        rects=detect_boxes(img_array, detect_fn,thresh,H,W)
#         rect_mask=np.array(list(map(func,rects)))>thresh
#         print(rects)
#         print(rect_mask)
#         valid_rects=rects[rect_mask]
        tracker.objects=tracker.update(frame,rects,thresh)
       
    else:
        t_rects=tracker.track_update(frame)
#         rect_mask=np.array(list(map(func,t_rects)))>thresh
#         print(rects)
#         print(rect_mask)
#         valid_rects=t_rects[rect_mask]  
#         print("tracked_valids: \n {}".format(valid_rects))
        tracker.objects=tracker.update(frame,t_rects,thresh)
            
    #Performing Liveness test for antispoofing and 
    #implementing face recognition per face tracked
    for i in tracker.objects.values():
        i.box_recognize(frame,class_model,label_map,tracker)
    image=visualize_boxes(frame,tracker.objects)
    y_stat=20
    for item,count in tracker.count.items():
        cv2.putText(image,'{}: {}'.format(item,count),(0,y_stat),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        y_stat+=20
    blk = np.zeros(image.shape, np.uint8)
    cv2.rectangle(blk, (int(0.45*W),0), (int(0.55*W), 337), (0, 255, 0), -1)
    res = cv2.addWeighted(image,1, blk, 0.25, 1)

    # Putting the image back to its position
    res=imutils.resize(res,width=1200)
    cv2.imshow('faces',res)
    fc+=1
    end=time.time()
    print("FPS: {}".format(1/(end-start)))
    FPS.append(int(1/(end-start)))
    print(tracker.objects.keys())
    if cv2.waitKey(10) & 0xff==ord('q'):
        print("STOPPING APPLICATION")
        
        break
print("MEAN FPS THROUGHOUT VIDEO: {}".format(np.array(FPS).sum()/len(FPS)))        
cap.release()
cv2.destroyAllWindows()

height: 337, width: 600
[[122 464 244 592]
 [ 98 492 135 600]]
[ True  True]
Valid_rects
 [[122 464 244 592]
 [ 98 492 135 600]]
FPS: 0.11244157027614227
odict_keys(['obj_0', 'obj_1'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 10.243701337397301
odict_keys(['obj_0'])
height: 337, width: 600
tracked_box
FPS: 12.374731885490394
odict_keys(['obj_0'])
height: 337, width: 600
tracked_box
FPS: 15.213952003714343
odict_keys([])
height: 337, width: 600
[[130 450 252 585]]
[ True]
Valid_rects
 [[130 450 252 585]]
FPS: 8.757491037480765
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 14.532774332143724
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 12.911549673848466
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 14.324710896783492
odict_keys([])
height: 337, width: 600
[[140 430 260 578]]
[ True]
Valid_rects
 [[140 430 260 578]]
FPS: 8.49493055077348
odict_keys(['obj_3'])
height: 337, width: 600
tracked_box
FPS: 14.12190285077456
odict_k

height: 337, width: 600
[[181   7 267 122]]
[False]
Valid_rects
 []
FPS: 9.37327001508464
odict_keys([])
height: 337, width: 600
FPS: 18.240545524127615
odict_keys([])
height: 337, width: 600
FPS: 18.230238966593358
odict_keys([])
height: 337, width: 600
FPS: 20.053567928091606
odict_keys([])
height: 337, width: 600
[[176   3 257 108]
 [238   0 278  85]]
[False False]
Valid_rects
 []
FPS: 10.34009392680612
odict_keys([])
height: 337, width: 600
FPS: 18.918315246317196
odict_keys([])
height: 337, width: 600
FPS: 18.23103136518534
odict_keys([])
height: 337, width: 600
FPS: 17.90509365981934
odict_keys([])
height: 337, width: 600
[[170   0 251 101]
 [229   0 269  71]]
[False False]
Valid_rects
 []
FPS: 9.82988546693259
odict_keys([])
height: 337, width: 600
FPS: 20.46261477065384
odict_keys([])
height: 337, width: 600
FPS: 19.671065837483937
odict_keys([])
height: 337, width: 600
FPS: 17.451761484914933
odict_keys([])
height: 337, width: 600
[[165   0 236  96]]
[False]
Valid_rects
 []
FP

FPS: 15.915850187834403
odict_keys(['obj_37'])
height: 337, width: 600
tracked_box
FPS: 16.17211995959191
odict_keys([])
height: 337, width: 600
[[128 311 305 509]]
[ True]
Valid_rects
 [[128 311 305 509]]
FPS: 8.952413070344132
odict_keys(['obj_38'])
height: 337, width: 600
tracked_box
FPS: 15.923584483033538
odict_keys(['obj_38'])
height: 337, width: 600
tracked_box
FPS: 16.43716395215776
odict_keys(['obj_38'])
height: 337, width: 600
tracked_box
FPS: 15.198570843615359
odict_keys([])
height: 337, width: 600
[[135 275 322 492]
 [171 454 277 534]]
[ True  True]
Valid_rects
 [[135 275 322 492]
 [171 454 277 534]]
FPS: 8.604194702464962
odict_keys(['obj_39', 'obj_40'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 13.5547612738096
odict_keys(['obj_39', 'obj_40'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 14.285776566757493
odict_keys(['obj_39', 'obj_40'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 14.531465236491636
odict_keys([])
height: 337, width: 600
[[14

tracked_box
FPS: 16.279581745212347
odict_keys(['obj_55'])
height: 337, width: 600
tracked_box
FPS: 15.198405623799689
odict_keys([])
height: 337, width: 600
[[137 458 251 574]
 [192  29 248 111]]
[ True False]
Valid_rects
 [[137 458 251 574]]
FPS: 8.873228233183553
odict_keys(['obj_56'])
height: 337, width: 600
tracked_box
FPS: 17.599536755357317
odict_keys(['obj_56'])
height: 337, width: 600
tracked_box
FPS: 15.419555019631487
odict_keys(['obj_56'])
height: 337, width: 600
tracked_box
FPS: 15.67401605405163
odict_keys([])
height: 337, width: 600
[[143 442 265 572]
 [189  34 241 105]]
[ True False]
Valid_rects
 [[143 442 265 572]]
FPS: 9.201203045785501
odict_keys(['obj_57'])
height: 337, width: 600
tracked_box
FPS: 16.44515542172454
odict_keys(['obj_57'])
height: 337, width: 600
tracked_box
FPS: 15.915970978415956
odict_keys(['obj_57'])
height: 337, width: 600
tracked_box
FPS: 16.029656919884278
odict_keys([])
height: 337, width: 600
[[155 423 280 567]]
[ True]
Valid_rects
 [[155 423

FPS: 14.744741809949343
odict_keys(['obj_79'])
height: 337, width: 600
tracked_box
FPS: 15.666756312565367
odict_keys([])
height: 337, width: 600
[[175 445 248 560]
 [191   6 267 121]
 [ 74 466 193 549]]
[ True False  True]
Valid_rects
 [[175 445 248 560]
 [ 74 466 193 549]]
FPS: 8.220692142301353
odict_keys(['obj_81', 'obj_82'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 14.122093042831217
odict_keys(['obj_81'])
height: 337, width: 600
tracked_box
FPS: 13.735783806442317
odict_keys(['obj_81'])
height: 337, width: 600
tracked_box
FPS: 15.26022732234075
odict_keys([])
height: 337, width: 600
[[ 78 449 205 544]
 [184 422 265 554]
 [188  12 259 117]]
[ True  True False]
Valid_rects
 [[ 78 449 205 544]
 [184 422 265 554]]
FPS: 8.520782376219923
odict_keys(['obj_83', 'obj_84'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 14.74526016783207
odict_keys(['obj_84'])
height: 337, width: 600
tracked_box
FPS: 15.273452919370461
odict_keys(['obj_84'])
height: 337, width: 600
track

...

height: 337, width: 600
[[109  28 197 109]
 [191  23 243 123]]
[False False]
Valid_rects
 []
FPS: 12.854725163583984
odict_keys([])
height: 337, width: 600
FPS: 20.065271990546947
odict_keys([])
height: 337, width: 600
FPS: 18.917206013016475
odict_keys([])
height: 337, width: 600
FPS: 18.378417222054257
odict_keys([])
height: 337, width: 600
[[189  26 238 123]
 [107  32 194 110]]
[False False]
Valid_rects
 []
FPS: 12.227935045625491
odict_keys([])
height: 337, width: 600
FPS: 22.788318709081526
odict_keys([])
height: 337, width: 600
FPS: 21.797764254049756
odict_keys([])
height: 337, width: 600
FPS: 19.293825411355577
odict_keys([])
height: 337, width: 600
[[107  34 191 111]
 [188  26 234 127]]
[False False]
Valid_rects
 []
FPS: 11.795901263594255
odict_keys([])
height: 337, width: 600
FPS: 23.317752229313527
odict_keys([])
height: 337, width: 600
FPS: 23.317881862404448
odict_keys([])
height: 337, width: 600
FPS: 23.33409735744089
odict_keys([])
height: 337, width: 600
[[185  36 231 

height: 337, width: 600
tracked_box
FPS: 15.915427435236797
odict_keys([])
height: 337, width: 600
FPS: 22.281327858140806
odict_keys([])
height: 337, width: 600
FPS: 23.890728061881273
odict_keys([])
height: 337, width: 600
[[137 373 229 485]]
[ True]
Valid_rects
 [[137 373 229 485]]
FPS: 12.378639687870757
odict_keys(['obj_608'])
height: 337, width: 600
tracked_box
FPS: 20.901812455461037
odict_keys(['obj_608'])
height: 337, width: 600
tracked_box
FPS: 20.47410169921751
odict_keys(['obj_608'])
height: 337, width: 600
tracked_box
FPS: 20.066807962988655
odict_keys([])
height: 337, width: 600
[[145 358 237 478]]
[ True]
Valid_rects
 [[145 358 237 478]]
FPS: 12.228077642956682
odict_keys(['obj_609'])
height: 337, width: 600
tracked_box
FPS: 20.463213769954333
odict_keys(['obj_609'])
height: 337, width: 600
tracked_box
FPS: 21.349404458922937
odict_keys(['obj_609'])
height: 337, width: 600
tracked_box
FPS: 17.903565086736783
odict_keys([])
height: 337, width: 600
[[153 340 247 466]]
[ Tr

height: 337, width: 600
[[179  83 224 175]]
[False]
Valid_rects
 []
FPS: 13.554586056011221
odict_keys([])
height: 337, width: 600
FPS: 23.31853004948018
odict_keys([])
height: 337, width: 600
FPS: 25.084950120810507
odict_keys([])
height: 337, width: 600
FPS: 24.474278778825507
odict_keys([])
height: 337, width: 600
[[176  85 222 176]]
[False]
Valid_rects
 []
FPS: 13.55541838090098
odict_keys([])
height: 337, width: 600
FPS: 23.73673040899599
odict_keys([])
height: 337, width: 600
FPS: 23.318011496936744
odict_keys([])
height: 337, width: 600
FPS: 22.80504567203132
odict_keys([])
height: 337, width: 600
[[173  92 220 177]]
[False]
Valid_rects
 []
FPS: 13.92586690040772
odict_keys([])
height: 337, width: 600
FPS: 25.0675591680612
odict_keys([])
height: 337, width: 600
FPS: 23.891816762933342
odict_keys([])
height: 337, width: 600
FPS: 23.33240989525097
odict_keys([])
height: 337, width: 600
[[168  99 217 175]]
[False]
Valid_rects
 []
FPS: 12.532843291052247
odict_keys([])
height: 337, 

[[ 83 426 130 533]
 [ 18 456 111 564]]
[ True  True]
Valid_rects
 [[ 83 426 130 533]
 [ 18 456 111 564]]
FPS: 11.796199840253793
odict_keys(['obj_617', 'obj_618'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 17.59060560308673
odict_keys([])
height: 337, width: 600
FPS: 22.787947277488616
odict_keys([])
height: 337, width: 600
FPS: 24.4554423117289
odict_keys([])
height: 337, width: 600
[[ 90 430 141 547]
 [ 22 459 118 571]]
[ True  True]
Valid_rects
 [[ 90 430 141 547]
 [ 22 459 118 571]]
FPS: 10.902139981233251
odict_keys(['obj_619', 'obj_620'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 18.23103136518534
odict_keys([])
height: 337, width: 600
FPS: 23.87318644665919
odict_keys([])
height: 337, width: 600
FPS: 23.87345821309245
odict_keys([])
height: 337, width: 600
[[ 95 431 149 535]
 [ 24 466 133 581]]
[ True  True]
Valid_rects
 [[ 95 431 149 535]
 [ 24 466 133 581]]
FPS: 9.734658116385022
odict_keys(['obj_621', 'obj_622'])
height: 337, width: 600
tracked_box
track

FPS: 9.198378009706502
odict_keys(['obj_663', 'obj_664', 'obj_665', 'obj_666'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
tracked_box
FPS: 12.378895424508672
odict_keys(['obj_663', 'obj_665'])
height: 337, width: 600
tracked_box
tracked_box
{'ep_green': 15, 'ep_black': 20}
FPS: 7.833290378488214
odict_keys(['obj_663', 'obj_665'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 14.538214639759863
odict_keys(['obj_663', 'obj_665'])
height: 337, width: 600
[[245 237 337 486]
 [129 427 222 553]
 [ 91 204 315 334]
 [ 97 478 173 600]
 [152 556 211 599]
 [106 456 193 583]]
[ True  True  True  True  True  True]
Valid_rects
 [[245 237 337 486]
 [129 427 222 553]
 [ 91 204 315 334]
 [ 97 478 173 600]
 [152 556 211 599]
 [106 456 193 583]]
FPS: 8.499440706816893
odict_keys(['obj_663', 'obj_665', 'obj_667', 'obj_668', 'obj_669', 'obj_670'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
tracked_box
tracked_box
tracked_box
FPS: 7.772584076437842
odict_keys(['obj_